# 차로변경 차량정보 추출
* 후행차량(`FV`)의 다음 정보를 추출함
    * `Vehicle ID` : 차량 ID
    * `Lane Identification` : 차량의 소속 차로
    * `Location X(m)` : X좌표
    * `Vehicle Veolcity` : 차량 속도 == $V_{FV}$ : 목표 차로 후행차량($FV$)의 속도
    * $D_{FV}$ : 대상차량($HV$)과 목표 차로 후행차량($FV$) 간의 초기 차간거리

# Set up

## Import

In [1]:
import pandas as pd
import numpy as np
import os

import warnings
from tqdm import tqdm

# 차로변경차량 정보 추출

## 양양방향

### Directory

In [2]:
file_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/27_남양주영업소_드론영상분석/양양방향/01_density_by_target_LOS_concated_lanechange'

In [3]:
save_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/38_22-10차 회의 준비/221022_FV_목표차로후행차량정보/차로변경차량정보'

In [4]:
file_list = os.listdir(file_dir)
file_list = file_list[3:]
print(file_list)

['04_1_A.csv', '04_2_A.csv']


### Extract
* 차로변경 차량별로 현재차로-목표차로 페어를 입력하기
* 현재차로-목표차로 페어에 따른 목표차로 후행차량(FV) 정보 구하기
* `df.at[idx, 'Lane Identification Past'] = str(Lane_past)`

In [5]:
warnings.filterwarnings(action = 'ignore') # pandas 경고메시지 숨기기

for file in file_list:
    
    file_path = os.path.join(file_dir, file)
    df = pd.read_csv(file_path)
    
    df2 = pd.DataFrame()
    
    # 차로변경을 한 차량 리스트만 뽑기
    LC_veh_list = df[df['Lane_change'] == 'Change']['Vehicle ID'].unique()
    
    for veh in LC_veh_list: # 차로변경을 한 각 차량별로
        df_veh = df[df['Vehicle ID'] == veh] ## 각 차량별 데이터프레임
        df_veh['Lane_record_split'] = df_veh['Lane_record_split'].backfill()
        df_veh['Lane_target'] = df_veh['Lane_record_split'].str.split('_', n = 1, expand = True)[1]
        
        df_veh['Lane_change_intend'] = df_veh['Lane_change_direction'].copy()
        df_veh['Lane_change_intend'] = df_veh['Lane_change_intend'].backfill()
    
        df2 = pd.concat([df2, df_veh])
    
    df2.dropna(subset = ['Lane_record_split'], inplace = True) # Lane_record_split이 nan인 행 드랍
    
    save_file = f'LC_vehicle_Lane_info_{file[:4]}.csv'
    save_path = os.path.join(save_dir, save_file)
    
    df2.to_csv(save_path)

## 서울방향

### Directory

In [6]:
file_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/27_남양주영업소_드론영상분석/서울방향/01_density_by_target_LOS_concated_lanechange'

In [7]:
file_list = os.listdir(file_dir)
file_list = file_list[:6]
print(file_list)

['01_1_A.csv', '01_2_A.csv', '01_3_A.csv', '02_1_A.csv', '02_2_A.csv', '02_3_A.csv']


### Extract
* 차로변경 차량별로 현재차로-목표차로 페어를 입력하기
* 현재차로-목표차로 페어에 따른 목표차로 후행차량(FV) 정보 구하기
* `df.at[idx, 'Lane Identification Past'] = str(Lane_past)`

In [8]:
warnings.filterwarnings(action = 'ignore') # pandas 경고메시지 숨기기

for file in file_list:
    
    file_path = os.path.join(file_dir, file)
    df = pd.read_csv(file_path)
    
    df2 = pd.DataFrame()
    
    # 차로변경을 한 차량 리스트만 뽑기
    LC_veh_list = df[df['Lane_change'] == 'Change']['Vehicle ID'].unique()
    
    for veh in LC_veh_list: # 차로변경을 한 각 차량별로
        df_veh = df[df['Vehicle ID'] == veh] ## 각 차량별 데이터프레임
        df_veh['Lane_record_split'] = df_veh['Lane_record_split'].backfill()
        df_veh['Lane_target'] = df_veh['Lane_record_split'].str.split('_', n = 1, expand = True)[1]
        
        df_veh['Lane_change_intend'] = df_veh['Lane_change_direction'].copy()
        df_veh['Lane_change_intend'] = df_veh['Lane_change_intend'].backfill()
    
        df2 = pd.concat([df2, df_veh])
    
    df2.dropna(subset = ['Lane_record_split'], inplace = True) # Lane_record_split이 nan인 행 드랍
        
    save_file = f'LC_vehicle_Lane_info_{file[:4]}.csv'
    save_path = os.path.join(save_dir, save_file)
    
    df2.to_csv(save_path)